In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import json
import psycopg2
from sodapy import Socrata
import re
from transform_dag import transformations_api

In [2]:
def read_db_water():
    with open('db_config.json') as file:
        db_config = json.load(file)

    engine = create_engine(f'postgresql+psycopg2://{db_config["user"]}:{db_config["password"]}@{db_config["host"]}:5433/{db_config["dbname"]}')

    water = pd.read_sql('SELECT * FROM water_table', engine)
    
    return water

In [13]:
from transform_dag import transformations_water

def transform_water():
    
    water = read_db_water()
    
    water = transformations_water(water)
    
    print(water.head())

# Llamar a la función principal para procesar los datos
transform_water()



2024-05-16 13:41:01,670 - INFO - Starting transformations on water data.
2024-05-16 13:41:01,775 - INFO - Dates converted successfully.
2024-05-16 13:41:02,148 - INFO - Standardized place names.
2024-05-16 13:41:02,191 - INFO - Scaled numerical columns.
2024-05-16 13:41:02,298 - INFO - Filtered top influential parameters.
2024-05-16 13:41:02,497 - INFO - Classified IRCA values into categories.
2024-05-16 13:41:08,601 - INFO - Categorized treatment data.
2024-05-16 13:41:17,363 - INFO - Critical Proportion.
2024-05-16 13:41:17,415 - INFO - Dropped unnecessary columns.
2024-05-16 13:41:17,417 - INFO - Column names standardized
2024-05-16 13:41:17,418 - INFO - Standardized column names.
2024-05-16 13:41:17,419 - INFO - All transformations applied successfully.


         año nombredepartamento  div_dpto nombremunicipio  divi_muni  \
0 2010-01-01            Bolívar        13        El Guamo      13248   
1 2010-01-01            Bolívar        13        El Guamo      13248   
2 2010-01-01            Bolívar        13        El Guamo      13248   
3 2010-01-01            Bolívar        13        El Guamo      13248   
4 2010-01-01            Bolívar        13        El Guamo      13248   

   ircaminimo  ircamaximo  ircapromedio nombreparametroanalisis2  \
0         0.0       100.0         37.32        Alcanilidad Total   
1         0.0       100.0         37.32                 Aluminio   
2         0.0       100.0         37.32                 Arsénico   
3         0.0       100.0         37.32                   Cadmio   
4         0.0       100.0         37.32                   Calcio   

   numeroparametrospromedio  is_top_20   rango_irca tratamientocategoría  \
0                         2       True  Riesgo alto  Tratamiento parcial   
1     

In [4]:
def extract_api(endpoint):
    try:
        # Inicializa el cliente de Socrata sin token de aplicación
        client = Socrata("www.datos.gov.co", None)

        # Obtiene los datos desde el endpoint especificado
        results = client.get(endpoint, limit=2000)

        # Convierte los resultados en un DataFrame
        api = pd.DataFrame.from_records(results)

        return api
    except Exception as e:
        print(f"Se produjo un error: {e}")

In [3]:
api = extract_api("tcwu-r53g")
print(api.head())

2024-05-06 13:53:34,873 - WARNING - Requests made without an app_token will be subject to strict throttling limits.


  fecha_terminacion_proyecto           fecha_de_corte  \
0    2018-12-31T00:00:00.000  2024-04-04T00:00:00.000   
1    2018-10-16T00:00:00.000  2024-04-04T00:00:00.000   
2    2018-10-08T00:00:00.000  2024-04-04T00:00:00.000   
3    2019-06-01T00:00:00.000  2024-04-04T00:00:00.000   
4    2019-02-12T00:00:00.000  2024-04-04T00:00:00.000   

  c_digo_divipola_departamento departamento c_digo_divipola_municipio  \
0                           18      CAQUETA       18029, 18205, 18610   
1                           15       BOYACA                     15693   
2                           17       CALDAS                     17050   
3                           17       CALDAS                     17446   
4                           05    ANTIOQUIA                      5250   

                                           municipio  \
0  Albania(CAQ), Curillo(CAQ), San Jose De Fragua...   
1                         Santa Rosa De Viterbo(BOY)   
2                                      Aranzazu(CA

In [5]:

def transform_api():
    
    pd.set_option('display.max_columns', None)  # Ninguna limitación en el número de columnas a mostrar
    pd.set_option('display.max_rows', None)  # Ninguna limitación en el número de filas a mostrar
    
    api = extract_api("tcwu-r53g")
    
    api = transformations_api(api)
    
    print(api.head())

transform_api()

2024-05-16 12:46:15,967 - WARNING - Requests made without an app_token will be subject to strict throttling limits.
2024-05-16 12:46:28,453 - INFO - Starting transformations on API data.
2024-05-16 12:46:28,463 - INFO - Elimination of parentheses within municipalities successfully.
2024-05-16 12:46:28,477 - INFO - Separates records with multiple municipalities into individual rows successfully.
2024-05-16 12:46:28,479 - INFO - Elimination of extra spaces and capitalization of each municipality name successful.
2024-05-16 12:46:28,490 - INFO - Dates converted successfully.
2024-05-16 12:46:28,500 - INFO - Text columns normalized successfully.
2024-05-16 12:46:28,502 - INFO - Number of municipalities computed successfully.
2024-05-16 12:46:28,506 - INFO - Regions mapped successfully.
2024-05-16 12:46:28,507 - INFO - Project financing calculated successfully.
2024-05-16 12:46:28,509 - INFO - Project duration calculated successfully.
2024-05-16 12:46:28,515 - INFO - Unnecessary columns dro

  fecha_terminacion_proyecto c_digo_divipola_departamento departamento  \
0                 2018-12-31                           18      CAQUETA   
0                 2018-12-31                           18      CAQUETA   
0                 2018-12-31                           18      CAQUETA   
1                 2018-10-16                           15       BOYACA   
2                 2018-10-08                           17       CALDAS   

  c_digo_divipola_municipio              municipio  \
0       18029, 18205, 18610                albania   
0       18029, 18205, 18610                curillo   
0       18029, 18205, 18610     san jose de fragua   
1                     15693  santa rosa de viterbo   
2                     17050               aranzazu   

                                           indicador  \
0       nueva población beneficiada acueducto urbano   
0       nueva población beneficiada acueducto urbano   
0       nueva población beneficiada acueducto urbano   
1     

In [32]:
api_done_df = pd.read_csv('C:/Users/JSLV3/Documents/5to Semestre/ETL/Proyecto Water Quality/ETL-Proyect/Main_Aqua_Quality/API/DATA/api_done.csv')

water_cleaned_df = pd.read_csv('C:/Users/JSLV3/Documents/5to Semestre/ETL/Proyecto Water Quality/water_cleaned.csv')

# Convertir columnas de fechas a formato datetime
api_done_df['fecha_terminacion_proyecto'] = pd.to_datetime(api_done_df['fecha_terminacion_proyecto'])
water_cleaned_df['año'] = pd.to_datetime(water_cleaned_df['año'])



# Filtrar el dataset de proyectos para incluir solo los años 2018 y 2019
api_done_filtered_df = api_done_df[api_done_df['fecha_terminacion_proyecto'].dt.year.isin([2018, 2019])]





# Renombrar la columna 'municipio' a 'nombre_municipio' y 'fecha_terminacion_proyecto' a 'fecha_proyecto'
api_done_filtered_df = api_done_filtered_df.rename(columns={'municipio': 'nombre_municipio', 'fecha_terminacion_proyecto': 'fecha_proyecto'})

# Función para limpiar y dividir nombres de municipios en el dataset de proyectos
def clean_and_split_municipios(municipios):
    # Eliminar texto entre paréntesis y caracteres especiales
    cleaned = re.sub(r'\(.*?\)', '', municipios).replace(',', '').strip().lower()
    # Dividir si hay múltiples municipios
    return [m.strip().title() for m in cleaned.split()]

# Aplicar la limpieza y división a los nombres de municipios en el dataset de proyectos
api_done_filtered_df['nombre_municipio'] = api_done_filtered_df['nombre_municipio'].apply(lambda x: ', '.join(clean_and_split_municipios(x)))
api_done_filtered_df = api_done_filtered_df.assign(nombre_municipio=api_done_filtered_df['nombre_municipio'].str.split(', ')).explode('nombre_municipio').reset_index(drop=True)

# Función para limpiar nombres de municipios en el dataset de calidad del agua
def clean_municipio_name_simple(name):
    return name.strip().title()

# Aplicar la limpieza a los nombres de municipios en el dataset de calidad del agua
water_cleaned_df['nombre_municipio'] = water_cleaned_df['nombre_municipio'].apply(clean_municipio_name_simple)




# Filtrar el dataset de calidad del agua para incluir solo las fechas de medición a partir de 2018
water_cleaned_filtered_df = water_cleaned_df[water_cleaned_df['año'].dt.year >= 2018]

# Realizar el merge de tipo "asof" usando los nombres de municipios normalizados
merged_df = pd.merge_asof(
    water_cleaned_filtered_df.sort_values('año'),
    api_done_filtered_df.sort_values('fecha_proyecto'),
    by='nombre_municipio',
    left_on='año',
    right_on='fecha_proyecto',
    direction='backward'
)

# Eliminar columnas redundantes excepto "fecha_de_corte"
columns_to_drop = ['c_digo_divipola_departamento', 'departamento', 'c_digo_divipola_municipio', 'fecha_de_corte', 'num_municipios']
merged_df = merged_df.drop(columns=columns_to_drop)

# Llenar las entradas faltantes
merged_df.fillna({
    'indicador': 'Ausencia de proyecto',
    'nombre_proyecto': 'Ausencia de proyecto',
    'origen': 'Ausencia de proyecto',
    'estado_seguimiento': 'Ausencia de proyecto',
    'región': 'Ausencia de proyecto',
    'aporte_nacion': -1,
    'contrapartida': -1,
    'total_financiamiento': -1,
    'fecha_proyecto': -1,
    'duracion_proyecto_dias': -1
}, inplace=True)

# Resultados
print(f"Municipios con proyectos: {merged_df[merged_df['nombre_proyecto'] != 'Ausencia de proyecto']['nombre_municipio'].nunique()}")
print(f"Número total de filas en el DataFrame resultante: {merged_df.shape[0]}")
print(f"Años disponibles en el DataFrame resultante: {merged_df['año'].dt.year.unique()}")
merged_df.to_csv('merged_water.csv', index=False)



Municipios con proyectos: 19
Número total de filas en el DataFrame resultante: 32355
Años disponibles en el DataFrame resultante: [2018 2019]


In [29]:
import pandas as pd
import re

# Cargar los datasets
api_done_df = pd.read_csv('C:/Users/JSLV3/Documents/5to Semestre/ETL/Proyecto Water Quality/ETL-Proyect/Main_Aqua_Quality/API/DATA/api_done.csv')
water_cleaned_df = pd.read_csv('C:/Users/JSLV3/Documents/5to Semestre/ETL/Proyecto Water Quality/water_cleaned.csv')

# Convertir columnas de fechas a formato datetime
api_done_df['fecha_terminacion_proyecto'] = pd.to_datetime(api_done_df['fecha_terminacion_proyecto'])
water_cleaned_df['año'] = pd.to_datetime(water_cleaned_df['año'])

# Filtrar el dataset de proyectos para incluir solo los años 2018 y 2019
api_done_filtered_df = api_done_df[api_done_df['fecha_terminacion_proyecto'].dt.year.isin([2018, 2019])]

# Renombrar la columna 'municipio' a 'nombre_municipio' y 'fecha_terminacion_proyecto' a 'fecha_proyecto'
api_done_filtered_df = api_done_filtered_df.rename(columns={'municipio': 'nombre_municipio', 'fecha_terminacion_proyecto': 'fecha_proyecto'})

# Función para limpiar y dividir nombres de municipios en el dataset de proyectos
def clean_and_split_municipios(municipios):
    cleaned = re.sub(r'\(.*?\)', '', municipios).strip().lower()
    municipios_list = [m.strip().title() for m in cleaned.split(',')]
    return municipios_list

# Aplicar la limpieza y división a los nombres de municipios en el dataset de proyectos
api_done_filtered_df['nombre_municipio'] = api_done_filtered_df['nombre_municipio'].apply(clean_and_split_municipios)
api_done_filtered_df = api_done_filtered_df.explode('nombre_municipio').reset_index(drop=True)

# Función para limpiar nombres de municipios en el dataset de calidad del agua
def clean_municipio_name_simple(name):
    return name.strip().title()

# Aplicar la limpieza a los nombres de municipios en el dataset de calidad del agua
water_cleaned_df['nombre_municipio'] = water_cleaned_df['nombre_municipio'].apply(clean_municipio_name_simple)

# Filtrar el dataset de calidad del agua para incluir solo las fechas de medición a partir de 2018
water_cleaned_filtered_df = water_cleaned_df[water_cleaned_df['año'].dt.year >= 2018]

# Asegurarse de que los datos estén ordenados por nombre de municipio y fecha de medición
water_cleaned_filtered_df = water_cleaned_filtered_df.sort_values(by=['nombre_municipio', 'año']).reset_index(drop=True)
api_done_filtered_df = api_done_filtered_df.sort_values(by=['nombre_municipio', 'fecha_proyecto']).reset_index(drop=True)

# Verificar el orden de las columnas 'nombre_municipio' y 'año'
print("Verificar el orden de la columna 'nombre_municipio' y 'año' en water_cleaned_filtered_df:")
print(water_cleaned_filtered_df[['nombre_municipio', 'año']].head(20))
print(water_cleaned_filtered_df[['nombre_municipio', 'año']].tail(20))

# Verificar el orden de las columnas 'nombre_municipio' y 'fecha_proyecto'
print("Verificar el orden de la columna 'nombre_municipio' y 'fecha_proyecto' en api_done_filtered_df:")
print(api_done_filtered_df[['nombre_municipio', 'fecha_proyecto']].head(20))
print(api_done_filtered_df[['nombre_municipio', 'fecha_proyecto']].tail(20))

# Mostrar municipios y años únicos para verificar el orden
print("Verificar municipios y sus años en water_cleaned_filtered_df:")
for municipio in water_cleaned_filtered_df['nombre_municipio'].unique():
    print(f"Municipio: {municipio}")
    print(water_cleaned_filtered_df[water_cleaned_filtered_df['nombre_municipio'] == municipio][['nombre_municipio', 'año']])

# Mostrar municipios y fechas únicos para verificar el orden
print("Verificar municipios y sus fechas en api_done_filtered_df:")
for municipio in api_done_filtered_df['nombre_municipio'].unique():
    print(f"Municipio: {municipio}")
    print(api_done_filtered_df[api_done_filtered_df['nombre_municipio'] == municipio][['nombre_municipio', 'fecha_proyecto']])

# Verificar el orden dentro de cada municipio
for municipio in water_cleaned_filtered_df['nombre_municipio'].unique():
    municipio_data = water_cleaned_filtered_df[water_cleaned_filtered_df['nombre_municipio'] == municipio]
    if not municipio_data['año'].is_monotonic_increasing:
        print(f"Problema de orden en municipio: {municipio}")
        print(municipio_data[['nombre_municipio', 'año']])

# Reordenar explícitamente los DataFrames
water_cleaned_filtered_df = water_cleaned_filtered_df.sort_values(by=['nombre_municipio', 'año']).reset_index(drop=True)
api_done_filtered_df = api_done_filtered_df.sort_values(by=['nombre_municipio', 'fecha_proyecto']).reset_index(drop=True)

# Verificar nuevamente si las claves están ordenadas
for municipio in water_cleaned_filtered_df['nombre_municipio'].unique():
    municipio_data = water_cleaned_filtered_df[water_cleaned_filtered_df['nombre_municipio'] == municipio]
    if not municipio_data['año'].is_monotonic_increasing:
        print(f"Problema persistente de orden en municipio: {municipio}")
        print(municipio_data[['nombre_municipio', 'año']])
        break  # Salir del bucle si se encuentra un problema

for municipio in api_done_filtered_df['nombre_municipio'].unique():
    municipio_data = api_done_filtered_df[api_done_filtered_df['nombre_municipio'] == municipio]
    if not municipio_data['fecha_proyecto'].is_monotonic_increasing:
        print(f"Problema persistente de orden en municipio: {municipio}")
        print(municipio_data[['nombre_municipio', 'fecha_proyecto']])
        break  # Salir del bucle si se encuentra un problema

# Realizar el merge de tipo "asof" usando los nombres de municipios normalizados
merged_df = pd.merge_asof(
    water_cleaned_filtered_df,
    api_done_filtered_df,
    by='nombre_municipio',
    left_on='año',
    right_on='fecha_proyecto',
    direction='backward'
)

# Llenar las entradas faltantes
merged_df.fillna({
    'indicador': 'Ausencia de proyecto',
    'nombre_proyecto': 'Ausencia de proyecto',
    'origen': 'Ausencia de proyecto',
    'estado_seguimiento': 'Ausencia de proyecto',
    'región': 'Ausencia de proyecto',
    'aporte_nacion': -1,
    'contrapartida': -1,
    'total_financiamiento': -1,
    'duracion_proyecto_dias': -1
}, inplace=True)

# Guardar el DataFrame resultante en un archivo CSV
merged_df.to_csv('merged_dataset.csv', index=False)

# Resultados
print(f"Municipios con proyectos: {merged_df[merged_df['nombre_proyecto'] != 'Ausencia de proyecto']['nombre_municipio'].nunique()}")
print(f"Número total de filas en el DataFrame resultante: {merged_df.shape[0]}")
print(f"Años disponibles en el DataFrame resultante: {merged_df['año'].dt.year.unique()}")


Verificar el orden de la columna 'nombre_municipio' y 'año' en water_cleaned_filtered_df:
   nombre_municipio        año
0         Abejorral 2018-01-01
1         Abejorral 2018-01-01
2         Abejorral 2018-01-01
3         Abejorral 2018-01-01
4         Abejorral 2018-01-01
5         Abejorral 2018-01-01
6         Abejorral 2018-01-01
7         Abejorral 2018-01-01
8         Abejorral 2018-01-01
9         Abejorral 2018-01-01
10        Abejorral 2018-01-01
11        Abejorral 2018-01-01
12        Abejorral 2018-01-01
13        Abejorral 2018-01-01
14        Abejorral 2018-01-01
15        Abejorral 2019-01-01
16        Abejorral 2019-01-01
17        Abejorral 2019-01-01
18        Abejorral 2019-01-01
19        Abejorral 2019-01-01
      nombre_municipio        año
32335    Zona Bananera 2018-01-01
32336    Zona Bananera 2018-01-01
32337    Zona Bananera 2018-01-01
32338    Zona Bananera 2018-01-01
32339    Zona Bananera 2018-01-01
32340    Zona Bananera 2019-01-01
32341    Zona Bananer

ValueError: left keys must be sorted